In [ ]:
import cv2
import numpy as np
import time
from tensorflow.keras.models import load_model

In [ ]:
# Load the pre-trained model
model = load_model('model_rafdb.h5')  # Specify the path to your model
CATAGORIES = ['Surprise', 'Fear', 'Disgust', 'Happy', 'Sad', 'Angry', 'Neutral']  # Model categories

In [ ]:
# Load the DNN face detection model
face_net = cv2.dnn.readNetFromCaffe('deploy.prototxt', 'res10_300x300_ssd_iter_140000.caffemodel')

In [ ]:
# IP webcam address
ip_camera_url = 'http://192.168.120.145:8080/video'
cap = cv2.VideoCapture(ip_camera_url)

In [ ]:
frame_skip = 10  # Process every 10th frame
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Problem receiving frames from the IP camera")
        break

    frame_count += 1
    if frame_count % frame_skip != 0:
        continue

    # Resize frame for faster processing
    frame = cv2.resize(frame, (1240, 720))
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Use DNN model for face detection
    h, w = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0), swapRB=False)
    face_net.setInput(blob)
    detections = face_net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.6:  # Minimum confidence for detection
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x1, y1) = box.astype("int")

            # Crop face from the frame
            face_image = frame_rgb[y:y1, x:x1]

            # Ensure the face was correctly extracted
            if face_image.shape[0] == 0 or face_image.shape[1] == 0:
                continue

            # Resize the face for the model
            face_image_resized = cv2.resize(face_image, (48, 48))
            face_image_resized = cv2.cvtColor(face_image_resized, cv2.COLOR_RGB2GRAY)
            face_image_resized = np.expand_dims(face_image_resized, axis=-1)
            face_image_resized = face_image_resized.astype('float32') / 255.0
            img_pred = np.expand_dims(face_image_resized, axis=0)

            # Predict emotions
            rslt = model.predict(img_pred)
            label = CATAGORIES[np.argmax(rslt)]
            confidence = np.max(rslt) * 100

            # Display bounding box and predicted label
            cv2.rectangle(frame, (x, y), (x1, y1), (0, 255, 0), 2)
            cv2.putText(frame, f'{label} ({confidence:.2f}%)', (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Show frame with predictions
    cv2.imshow('IP Camera Face Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()